In [1]:
# Imports and definitions
import numpy as np
from collections import defaultdict
from collections import namedtuple
import urllib.request
import hashlib

In [8]:
HashPointer = namedtuple('HashPointer', ['hash', 'pointer'])
Transaction = namedtuple('Transaction', ['payer', 'payee', 'amount'])
Puzzle = namedtuple('Puzzle', ['difficulty', 'denominator', 'numerator'])

class Block:
    def __init__(self, transaction, prev, nonce):
        self.transaction = transaction
        self.prev = prev
        self.nonce = nonce
    
    def __repr__(self):
        return f'\nBlock(\n transaction: {self.transaction},\n nonce: {self.nonce},\n prev: {self.prev})'
    
class Blockchain:
    def __init__(self, payer, payee, amount):
        first_transaction = Transaction(payer, payee, amount)
        self.blockchain = Block(first_transaction, None, None)
        self.users = {}
        self.puzzle = self.new_puzzle()
        
        return_value = self.process_transaction(first_transaction)
        assert return_value == ''
        
        return self.blockchain
    
    def new_user(self):
        q = 2**32
        p = q**3
        t = 2**32
        
        sk = random.randint(0, t-1)
        a = random.randint(1, q-1)
        e = random.randint(-5, 5) % q
        pk = (-(a*sk+e)%q, a)
        
        return sk, pk
    
    
#     def sign(self, sk, m):
#         q = 2**32
#         p = q**3
#         t = 2**32
        
#         p0, p1 = pk
#         u = random.randint(0, t-1)
#         e1 = random.randint(-5, 5) % q
#         e2 = random.randint(-5, 5) % q
#         Delta = int(q / t)
#         ct1 = (p0*u + e1 + Delta*m) % q
#         ct2 = (p1 * u + e2) % q
#         return (ct1, ct2)
    
    
#     def verify_signature(self, pk, ct, m):
#         c0, c1 = ct
#         m = round(t * ((c0 + c1 * pk) % q) / q) % t
#         return m
        
        
    def process_transaction(self, transaction):
        users = self.users
        payer = transaction.payer
        payee = transaction.payee
        amount = transaction.amount
        
        return_value = ''
        
        if payer == None and payee != None:
            if payee not in users.keys():
                users[payee] = amount
            else:
                return_value = 'This user already exists in the system.'
        elif payer != None and payee != None:
            if payee in users.keys() and payer in users.keys():
                payer_balance = users[payer]
                payee_balance = users[payee]
                
                if payer_balance - amount < 0:
                    return_value += 'Payer does not have enough currency. '
                else:
                    users[payer] = payer_balance - amount
                    users[payee] = payee_balance + amount
                    
            else:
                if payee not in users.keys():
                    return_value += 'Payee does not exist. '
                if payer not in users.keys():
                    return_value += 'Payer does not exist. '
        elif payee == None:
            return_value += 'Payee not specified. '   
                
        self.users = users
        return return_value
        
        
    def get_blockchain(self):
        return self.blockchain

    
    # Creates a block with the given transaction
    def add_transaction(self, transaction, nonce):
        prev_hash = hashlib.sha256(bytes(str(self.blockchain), encoding='utf-8')).hexdigest()
        prev = HashPointer(prev_hash, self.blockchain)
        new_block = Block(transaction, prev, nonce)
        self.blockchain = new_block

        block_hash = hashlib.sha256(bytes(str(new_block), encoding='utf-8')).hexdigest()

        return new_block, block_hash
    
    
    def new_puzzle(self):
        if self.puzzle != None:
            denominator = self.puzzle.denominator
        else:
            denominator = 1      
        
        hash_string = hashlib.sha256(bytes('hello', encoding='utf-8')).digest()
        denominator = np.randint(denominator, denominator**2)
        numerator = 2**(len(hash_string) * 8)
        difficulty = int(numerator/denominator)
        self.puzzle = Puzzle(difficulty, denominator, numerator)
        
        
    def mine(self, transaction):
        blockchain = self.blockchain
        nonce = 0
        finished = False
        
        while not finished:
            nonce = nonce + 1
            new_blockchain, final_hash = self.add_transaction(transaction, blockchain, nonce)
            
            if self.verify_puzzle_solution(final_hash):
                self.blockchain = new_blockchain
                self.puzzle = self.new_puzzle()
                finished = True
                
        
    def verify_puzzle_solution(self, final_hash):
        difficulty = self.puzzle.difficulty
        
        if int(final_hash, 16) <= difficulty:
            return True
        return False
        

    def get_puzzle(self):
        return self.puzzle
    

In [9]:
luke = blockchain.new_user()

Blockchain(None, 'Luke', 10)



4.0
